In [1]:
import subprocess
import utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

mapDict={0:["corners",4],
         1:["corners2",5],
         2:["loop-chain",7],
         3:["connector",6],
         4:["string",5],
         5:["tree",3],
         6:["tree2",17],
         7:["tunnel",4],
         8:["lak303d",1000],
         9:["random-32-32-10-2",485],
         10:["random-32-32-10",460],
         11:["maze-128-128-1",350],
         12:["room-64-64-8",1000],
         13:["w_woundedcoast",1000],
         14:["Berlin_1_256",1000],
         15:["brc202d",1000],
         16:["den312d",400],
         17:["maze-32-32-4",100],
         18:["maze-128-128-2",300],
         19:["orz900d",1000],
         20:["warehouse-20-40-10-2-1",1000],
         21:["warehouse-20-40-10-2-2",1000]}

Candidates=[8,10,11,12,13,14,15,16,17,18,19,20,21]

读取csv文件

In [13]:
import random
from tqdm import tqdm

df = pd.read_csv('result.csv')
# 提取特定列
columns = ['instance','total', 'n0', 'ub0', 'n4', 'ub4', 'average_ub0', 'average_ub4']
# 打印提取的 DataFrame
print(df)
TieBreakers=['0','4']
selected=11
num=200
instance=mapDict[selected][0]+"&&"+str(num)
df=utils.check_and_add_instance(df,instance)
for i in tqdm(range(10), desc="Processing"):
    seed=random.randint(0,100000000)
    # print(mapDict[selected][0])
    # print(num)
    # print(seed)
    df=utils.df_add_num(df,instance,'total',1)
    for tiebreaker in TieBreakers:
        # 构建命令
        compute = [
            '../build/main',
            '-i', f'../assets/{mapDict[selected][0]}.scen',
            '-m', f'../assets/{mapDict[selected][0]}.map',
            '-N', str(num),
            '-v', '1',
            '-t', '5',
            '--no-star',
            '--tie-breaker',tiebreaker,
            '-s',str(seed),
        ]
        try:
            # 运行命令
            result_compute = subprocess.run(compute, capture_output=True, text=True, check=True)
            # 打印输出
            # print("标准输出:")
            # print(result_compute.stdout)
            # print("标准错误:")
            # print(result_compute.stderr)
            cost,lb,ub=utils.extract_total_cost(result_compute.stdout)
            if ub>=1:
                df=utils.df_add_num(df,instance,'n'+tiebreaker,1)
                df=utils.df_add_num(df,instance,'ub'+tiebreaker,round(ub,5))
                average_ub=df.loc[df['instance'] == instance, 'ub'+tiebreaker]/df.loc[df['instance'] == instance, 'n'+tiebreaker]
                df=utils.df_set_num(df,instance,'average_ub'+tiebreaker,round(average_ub,5))
                df.to_csv('result.csv', index=False)
                pass
        except subprocess.CalledProcessError as e:
            print(f"命令执行失败: {e}")
            print(f"返回码: {e.returncode}")
            print(f"标准错误: {e.stderr}")
            break
    # print(df)
print(df)


               instance  total  n0        ub0  n4        ub4  average_ub0  \
0  random-32-32-10&&400     51  51   96.49768  51   93.67884      1.89211   
1   maze-128-128-1&&300     51  47  135.00067  50  142.42014      2.87235   

   average_ub4  
0      1.83684  
1      2.84840  


Processing: 100%|██████████| 10/10 [00:10<00:00,  1.06s/it]

               instance  total  n0        ub0  n4        ub4  average_ub0  \
0  random-32-32-10&&400     51  51   96.49768  51   93.67884      1.89211   
1   maze-128-128-1&&300     51  47  135.00067  50  142.42014      2.87235   
2   maze-128-128-1&&200     10  10   19.75819  10   17.97566      1.97582   

   average_ub4  
0      1.83684  
1      2.84840  
2      1.79757  
